In [1]:
import sys
import os
import torch
import torch.nn as nn
os.chdir(os.path.dirname(os.path.realpath(".")))
sys.path.insert(0, '..')
print(os.path.realpath("."))

/Users/yuxinwang/Desktop/FidelityFusion/MF_BayesianOptimization


## MF Acquisition Function Illustration file
在这个notebook中，我将讲述几种多精度的贝叶斯优化中采集函数的数学方法以及代码实现，分别是MF_UCB，MF_EI， Fabolas以及cfKG。
<!-- ![jupyter](./assets/figures/Summary_of_MFBO_lit.png) -->
<!-- <img src="./assets/figures/Summary_of_MFBO_lit.png", width=320, heigth=240> -->
对于采集函数来说，我们尽可能地简化他们的依赖，努力将输入控制为后验分布的均值函数，后验分布的方差函数 以及 反应采集不同精度数据的成本差异的cost函数。

### MF_UCB
Based on paper "Gaussian Process Bandit Optimisation with Multi-fidelity Evaluations" by Kirthevasan Kandasamy, Gautam Dasarathy (link: https://proceedings.neurips.cc/paper_files/paper/2016/file/605ff764c617d3cd28dbbdd72be8f9a2-Paper.pdf)

In MF_UCB, the method are divided into two parts: 
<!-- 1. 用高低精度分别做UCB，记为UCB_l&UCB_h。min(UCB_l, UCB_h)作为MF-UCB，选择最大值；
2. 人为设置精度判断指标，其中beta_t和gamma为作者设定的，sigma是x_t点的方差。如果满足下式选低精度，ortherwise选高精度 -->

1. Implement UCB using low precision and high precision separately, denoted as UCB_l and UCB_h. Consider the minimum of UCB_l and UCB_h as MF-UCB, and choose the maximum value.
    <!-- $$ \phi_t^{(m)}(x) = \mu_{t-1}^{(m)}(x) + \beta_t^{1/2}\sigma_{t-1}^{(m)}(x) $$ -->

    \begin{aligned}
    \phi_t^{(m)}(x) &= \mu_{t-1}^{(m)}(x) + \beta_t^{1/2}\sigma_{t-1}^{(m)}(x), \\
    \phi_t(x) &= \min_m \phi_t^{(m)}(x), \\
    x_t &= \argmax_{x\in\mathcal{X}}\phi_t^{(m)}(x), \\
    \end{aligned}
    where $m$ denotes the $m^{th}$ fidelity, $\beta_t$ coefficient trading off exploration and exploitation in $t$-th interation, $\phi_t^{(m)}(x)$ denotes the upper confidence bound (UCB) and $\phi_t(x)$ denotes the combined UCB provided by all fidelities.
    
    Choice of $\beta_t$, the paper use $\beta_t = 0.2 d \log(2t)$ where $t$ means $t$-th interation and $d$ means the dimension of $x$.

2. Establish precision evaluation criteria based on manually set indicators, where $\beta_t$ and $\gamma$ are predefined by the author, and sigma represents the variance at point $x_t$. If the following condition is met, $\gamma^{(m)}$ is the parameter in MF_UCB for switching from the $m$th fidelity to the $(m+1)$th fidelity. This parameter is setting by author.


In [ ]:
def UCB_compute_x(x, mean_function, var_function, BO_iteration_count, fidelity_num):    
    N_UCB = []
    UCB_x = []
    beta = 0.2 * int(x.shape[0]) * torch.log(BO_iteration_count)
    for i in range(fidelity_num):
        torch.manual_seed(2024)
        x = nn.Parameter(torch.from_numpy(torch.rand(1, x.shape[0])).double())
        # optimise_adam(fidelity_indicator=i+1, niteration=15, lr=0.01)
        UCB_x.append(x.detach())
        mean = mean_function(x, i+1)
        var = var_function(x, i+1)
        UCB = mean + beta * var
        N_UCB.append(UCB)

    new_x = UCB_x[N_UCB.index(min(N_UCB))]

    return new_x

def UCB_compute_s(x, mean_function, var_function, fidelity_num):
    beta = 0.2 * int(x.shape[0])
    gamma = 0.1
    fidelity_list = []
    for i in range(fidelity_num):
        mean = mean_function(x, i+1)
        var = var_function(x, i+1)
        UCB = mean + beta * var
        if UCB > gamma:
            fidelity_list.append(i)

    new_s = min(fidelity_list)

    return new_s

### MF_EI
Based on paper "Gaussian Process Bandit Optimisation with Multi-fidelity Evaluations" by Kirthevasan Kandasamy, Gautam Dasarathy (link: https://proceedings.neurips.cc/paper_files/paper/2016/file/605ff764c617d3cd28dbbdd72be8f9a2-Paper.pdf)

